In [10]:
# Kütüphanelerin yüklenmesi
import numpy as np, pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as split
from sklearn.preprocessing import StandardScaler as SS
%matplotlib notebook


AttributeError: module 'numpy' has no attribute '__config__'

In [ ]:

def PM10_hci(PM10):
    PM10_index=0
    if PM10>=0 and PM10<=50:
        PM10_index=1 # İyi
    elif PM10>=51 and PM10<=100:
        PM10_index=2 # Orta
    elif PM10>=101 and PM10<=260:
        PM10_index=3 # Hassas
    elif PM10>=261 and PM10<=400:
        PM10_index=4 # Sağlıksız
    elif PM10>=401 and PM10<=520:
        PM10_index=5 # Kötü
    elif PM10>=521:
        PM10_index=6 # Tehlikeli
    return PM10_index
        
def SO2_hci(SO2):
    SO2_index=0
    if SO2>=0 and SO2<=100:
        SO2_index=1
    elif SO2>=101 and SO2<=250:
        SO2_index=2
    elif SO2>=251 and SO2<=500:
        SO2_index=3
    elif SO2>=501 and SO2<=850:
        SO2_index=4
    elif SO2>=851 and SO2<=1100:
        SO2_index=5
    elif SO2>=1101:
        SO2_index=6
    return SO2_index

def CO_hci(CO):
    CO_index=0
    if CO>=0 and CO<=5500:
        CO_index=1
    elif CO>=5501 and CO<=10000:
        CO_index=2
    elif CO>=10001 and CO<=16000:
        CO_index=3
    elif CO>=16001 and CO<=24000:
        CO_index=4
    elif CO>=24001 and CO<=32000:
        CO_index=5
    elif CO>=32001:
        CO_index=6
    return CO_index

def NO2_hci(NO2):
    NO2_index=0
    if NO2>=0 and NO2<=100:
        NO2_index=1
    elif NO2>=101 and NO2<=200:
        NO2_index=2
    elif NO2>=201 and NO2<=500:
        NO2_index=3
    elif NO2>=501 and NO2<=1000:
        NO2_index=4
    elif NO2>=1001 and NO2<=2000:
        NO2_index=5
    elif NO2>=2001:
        NO2_index=6
    return NO2_index
    
def O3_hci(O3):
    O3_index=0
    if O3>=0 and O3<=120:
        O3_index=1
    elif O3>=121 and O3<=160:
        O3_index=2
    elif O3>=161 and O3<=180:
        O3_index=3
    elif O3>=181 and O3<=240:
        O3_index=4
    elif O3>=241 and O3<=700:
        O3_index=5
    elif O3>=701:
        O3_index=6
    return O3_index


# verisetinin yüklenmesi
df=pd.read_excel('konya.xlsx', header=None, names=["Tarih", "PM10(µg/m³)", "SO2(µg/m³)", "CO(µg/m³)", "O3(µg/m³)"])
df=df.drop([0, 1], axis=0)


# Eksik Verilerin Tamamlanması
data=df.fillna(df.mean())

# Hava Kalite İndexinin Oluşturulması
dataindex=pd.DataFrame(data, columns=["Index"])

for i in range(len(data)):
    pm10=PM10_hci(data.iloc[i]["PM10(µg/m³)"])
    so2=SO2_hci(data.iloc[i]["SO2(µg/m³)"])
    co=NO2_hci(data.iloc[i]["CO(µg/m³)"])
    o3=O3_hci(data.iloc[i]["O3(µg/m³)"])
    dataindex.iloc[i]["Index"]=max(pm10,so2,co,o3)

"""
#Verinini Görselleştirilmesi
import matplotlib.pyplot as plt 
plt.plot(data["Tarih"],data["PM10(µg/m³)"],color="g")
plt.title("PM10 Grafiği")
plt.ylabel("PM10")
plt.xlabel("Tarih")
plt.legend(["PM10"], loc="upper left")
plt.show()

plt.plot(data["Tarih"],data["SO2(µg/m³)"],color="g")
plt.title("SO2 Grafiği")
plt.ylabel("SO2")
plt.xlabel("Tarih")
plt.legend(["SO23"], loc="upper left")
plt.show()
 
plt.plot(data["Tarih"],data["CO(µg/m³)"],color="g")
plt.title("CO Grafiği")
plt.ylabel("CO")
plt.xlabel("Tarih")
plt.legend(["CO"], loc="upper left")
plt.show()
 
plt.plot(data["Tarih"],data["O3(µg/m³)"],color="g")
plt.title("O3 Grafiği")
plt.ylabel("O3")
plt.xlabel("Tarih")
plt.legend(["O3"], loc="upper left")
plt.show()
"""
# Verisetine index sütununun eklenmesi
df=pd.concat([data, dataindex.Index.astype(int)],axis =1)

#Sınıf sayısı ve etiketlerinin belirlenmesi
label_encoder=LabelEncoder().fit(df.Index)
labels=label_encoder.transform(df.Index)
classes=list(label_encoder.classes_) # [ 1,2,3,4,5,6]

#Girdi ve çoktiların hazırlanması
X=df.drop(["Tarih","Index"],axis=1)
y=labels
nb_features=4
nb_classes=len(classes)

# Değerlerin standartlaştırılması
X=df.drop(["Tarih","Index"],axis=1)
X=SS().fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, = train_test_split(X,y,test_size = 0.2)

#çıktı değerlerinin kategorileştirilmesi
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Girdi verilerinin yeniden boyutlandırılması
X_train=np.array(X_train).reshape(X_train.shape[0],X_train.shape[1],1)
X_test=np.array(X_test).reshape(X_test.shape[0],X_test.shape[1],1)



In [ ]:
#modelin oluşturulması
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Activation,Dropout
from tensorflow.python.keras.layers import Flatten,LSTM,BatchNormalization
model= Sequential()
model.add(LSTM(512,input_shape=(nb_features,1)))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add((Flatten()))
model.add(Dropout(0.15))
model.add(Dense(2048, activation="relu"))
model.add(Dense(1024, activation="relu"))
model.add(Dense(nb_classes, activation="softmax"))
model.summary()

#modelin derlenmesi
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])


In [ ]:
#modelin eğitilmesi
score = model.fit(X_train,y_train, epochs = 100, validation_data=(X_test,y_test))


In [1]:
print(("Ortalama Eğitim Kaybı:" , np.mean(model.history.history["loss"])))
print(("Ortalama Eğitim Başarımı:" , np.mean(model.history.history["accuracy"])))
print(("Ortalama Doğrulama Kaybı:" , np.mean(model.history.history["val_loss"])))
print(("Ortalama Doğrulama Başarımı:" , np.mean(model.history.history["val_accuracy"])))


NameError: name 'np' is not defined

In [ ]:
#Eğitim ve doğrulama değerlerinin grafik üzerinde gösterilmesi
import matplotlib.pyplot as plt 
plt.plot(model.history.history["accuracy"],color="b")
plt.plot(model.history.history["val_accuracy"],color="r")
plt.title("Model Başarımları")
plt.ylabel("Başarım")
plt.xlabel("Epok Sayısı")
plt.legend(["Eğitim","Test"], loc="upper left")
plt.show()


plt.plot(model.history.history["loss"],color="g")
plt.plot(model.history.history["val_loss"],color="r")
plt.title("Model kayıplar")
plt.ylabel("Kayıp")
plt.xlabel("Epok Sayısı")
plt.legend(["Eğitim","Test"], loc="upper left")
plt.show()